This notebook uses the Dept. Ag, Water and the Environment's data on Species of National Environmental Significance
See https://environment.gov.au/science/erin/databases-maps/snes
and http://www.environment.gov.au/fed/catalog/search/resource/details.page?uuid=%7B337B05B6-254E-47AD-A701-C55D9A0435EA%7D

Specifically a subset of the "critically endangered" mammals listed in the dataset - Leadbetter Possum

# Install requisite software libraries

In [32]:
!pip install --quiet git+https://github.com/CSIRO-enviro-informatics/AusPIX_DGGS.git

In [ ]:
!pip install geojson pygeoj

Load the "critically_endangered_mammals_leadbetter.geojson" data file

In [ ]:
import geojson
fname = "critically_endangered_mammals_leadbetter.geojson"
geojson_data = None
with open(fname) as json_file:
    geojson_data = geojson.load(json_file)
    

Show the locations of critically endangered leadbetter possums on a map

In [ ]:
#draw the geometry for the feature on a map
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker

x_coord = -37.8
y_coord = 145.89

map1 = ipy.Map(center=[x_coord, y_coord], zoom=8)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=geojson_data, style = {'color': 'red', 'opacity':0.8, 'weight':1.9, 'fillOpacity':0.5})
map1.add_layer(geo_json1)


In [ ]:
## Critically endangered mammals: leadbetter Possum
map1

In [ ]:
from pandas import json_normalize
features = geojson_data['features']
json_normalize(features)

In [ ]:
from auspixdggs.callablemodules.dggs_in_poly_for_geojson_callable import cells_in_poly
from auspixdggs.callablemodules.dggs_in_line import line_to_DGGS, densify_my_line
import geojson
from geojson.utils import coords
from shapely.geometry import shape, LineString, MultiLineString, Polygon, MultiPolygon

def get_geojson_by_file(fname):
    data = None
    with open(fname) as json_file:
        data = geojson.load(json_file)
    return data

def geojson_to_shape(g):
    return shape(g)
    
def bbox(coord_list):
     box = []
     for i in (0,1):
         res = sorted(coord_list, key=lambda x:x[i])
         box.append((res[0][i],res[-1][i]))
     #ret = f"({box[0][0]} {box[1][0]}, {box[0][1]} {box[1][1]})"
     ret = [ box[0][0], box[1][0], box[0][1], box[1][1] ]
     #print("BBOX {}".format(ret))
     return ret

def get_cells_in_geojson(geojson, resolution, cell_obj=True):
    list_cells = []
    #print("Type geojson: {}".format(type(geojson)))
    dict_cells = {}
    for fea in geojson['features']:  # for feature in attribute table
        #print(fea)
        res_cells = get_cells_in_feature(fea, resolution, cell_obj=cell_obj)
        print(res_cells)
        for item in res_cells:
            print("'{}'".format(str(item)))
        #cell_list = [item[0] for item in res_cells]
        cell_list = res_cells
        #cell_id_list = res_cells
        for cell in cell_list:
            cell_id = str(cell)
            if cell_id not in dict_cells:
                dict_cells[cell_id] = cell
    list_cells = list(dict_cells.values())
    return list_cells 

def get_cells_in_feature(fea, resolution, cell_obj=True):
    geom = geojson_to_shape(fea['geometry'])
    curr_coords = list(coords(fea))
    thisbbox = bbox(curr_coords)
    cells = []
    if isinstance(geom, LineString) or isinstance(geom, MultiLineString): 
        fea['geometry']['coordinates'] = densify_my_line(fea['geometry']['coordinates'], resolution)
        curr_coords = list(coords(fea))
        print(curr_coords)        
        res_cells = line_to_DGGS(curr_coords, resolution)  # start at DGGS level 10   
        cells = [str(item) for item in res_cells]
    elif isinstance(geom, Polygon) or  isinstance(geom, MultiPolygon):
        res_cells = cells_in_poly(thisbbox, fea['geometry']['coordinates'], resolution, return_cell_obj=cell_obj)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]
    else: #try something anyway
        res_cells = cells_in_poly(thisbbox, fea['geometry']['coordinates'], resolution, return_cell_obj=cell_obj)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]
    return cells   

# Use the Geoscience Australia AusPIX DGGS Engine to DGGS-enable the data

Use the DGGS Engine to find cells that match the features. Also specify the DGGS resolution needed.

In [ ]:
DGGS_RESOLUTION=7

In [ ]:
list_cells = get_cells_in_geojson(geojson_data, DGGS_RESOLUTION, True)  

In [ ]:
from auspixdggs.callablemodules.dggs_in_poly_for_geojson_callable import cells_in_poly, get_dggs_cell_bbox
from geojson import Feature, FeatureCollection

list_features = []
for cell in list_cells:
    bbox_coords = get_dggs_cell_bbox(cell)
    geom_obj = Polygon(bbox_coords)
    feat = Feature(geometry=geom_obj, properties={"dggs_cell_id": str(cell)}) 
    list_features.append(feat)

feature_collection = FeatureCollection(list_features)

Show the matching DGGS Cells on a map

In [ ]:
map2 = ipy.Map(center=[x_coord, y_coord], zoom=9)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json2 = GeoJSON(data=feature_collection, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'blue', 
                          'fillOpacity': 0.2})
map2.add_layer(geo_json1)
map2.add_layer(geo_json2)



In [ ]:
# DGGS Enabled version - Critically endangered mammals: leadbetter Possum

map2


# Query the Loc-I Integration API for matching LGAs

In [ ]:
import requests
import json
#iterate over features in feature collection and query Loc-I API for features
def find_features_at_dggs_cell(dggs_cell_id):
    payload = {
        'dggs_cell' : dggs_cell_id
    }
    r = requests.get('https://api.loci.cat/api/v1/location/find-at-DGGS-cell', params=payload)
    json_res = r.json()
    return json_res


In [ ]:
target_feature_type = 'asgs16_lga'

set_of_lga_features = set([])
#print(feature_collection)
for feature in feature_collection['features']:
    cell_id = feature['properties']['dggs_cell_id']
    #pad it out
    cell_id = cell_id + "000"
    print("Processing {}".format(cell_id))
    res = find_features_at_dggs_cell(cell_id)
    if 'locations' in res:
        for loc in res['locations']:
            if 'dataType' in loc and loc['dataType'] == target_feature_type:
                set_of_lga_features.add(loc['uri'])
                print("Adding {}".format(loc['uri']))

    


In [ ]:
def get_geom(loci_uri):
    payload = {
        "uri": loci_uri
    }
    url = "https://api.loci.cat/api/v1/location/geometry"
    r = requests.get(url, params=payload)
    res = r.json()
    #get the first geom result
    geojson_data = []
    if len( res['geometry']) > 0:
        geojson_data = res['geometry'][0]
    return geojson_data



Show the matching LGAs on a map

In [ ]:
map3 = ipy.Map(center=[x_coord, y_coord], zoom=7)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))


In [ ]:
for feature_uri in list(set_of_lga_features):
    geojson_data = get_geom(feature_uri)
    curr_geojson_layer = GeoJSON(data=geojson_data, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'yellow', 
                          'fillOpacity': 0.3})
    map3.add_layer(curr_geojson_layer)
map3.add_layer(geo_json1)
map3.add_layer(geo_json2)


In [ ]:
map3

# Try a different Loc-I Feature - Query the Loc-I Integration API for matching ASGS Statistical Area Level 2 locations

In [ ]:
target_feature_type = 'asgs16_sa2'

set_of_features = set([])
#print(feature_collection)
for feature in feature_collection['features']:
    cell_id = feature['properties']['dggs_cell_id']
    #pad it out
    cell_id = cell_id + "000"
    print("Processing {}".format(cell_id))
    res = find_features_at_dggs_cell(cell_id)
    if 'locations' in res:
        for loc in res['locations']:
            if 'dataType' in loc and loc['dataType'] == target_feature_type:
                set_of_features.add(loc['uri'])
                print("Adding {}".format(loc['uri']))


Show the matching ASGS SA2 on a map

In [ ]:

map4 = ipy.Map(center=[x_coord, y_coord], zoom=8)


for feature_uri in list(set_of_features):
    geojson_data = get_geom(feature_uri)
    curr_geojson_layer = GeoJSON(data=geojson_data, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'green', 
                          'fillOpacity': 0.3})
    map4.add_layer(curr_geojson_layer)

    
map4.add_layer(geo_json1)
map4.add_layer(geo_json2)

In [ ]:
map4

# Try a different Loc-I Feature-type again - Query the Loc-I Integration API for matching ASGS State Suburbs

In [ ]:
target_feature_type = 'asgs16_ssc'

set_of_features = set([])
#print(feature_collection)
for feature in feature_collection['features']:
    cell_id = feature['properties']['dggs_cell_id']
    #pad it out
    cell_id = cell_id + "000"
    print("Processing {}".format(cell_id))
    res = find_features_at_dggs_cell(cell_id)
    if 'locations' in res:
        for loc in res['locations']:
            if 'dataType' in loc and loc['dataType'] == target_feature_type:
                set_of_features.add(loc['uri'])
                print("Adding {}".format(loc['uri']))


Show the matching State Suburbs on a map

In [ ]:

map5 = ipy.Map(center=[x_coord, y_coord], zoom=8)


for feature_uri in list(set_of_features):
    geojson_data = get_geom(feature_uri)
    curr_geojson_layer = GeoJSON(data=geojson_data, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'purple', 
                          'fillOpacity': 0.3})
    map5.add_layer(curr_geojson_layer)

    
map5.add_layer(geo_json1)
map5.add_layer(geo_json2)

In [ ]:
map5